## Data preprocessing

In [67]:
# Import libraries
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

from google.colab import drive

In [68]:
# Connect drive
drive.mount('/content/gdrive')
drive_path = '/content/gdrive/MyDrive/Boltzmann_machines'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [79]:
# Load sets
# Paths
movies_file = os.path.join(drive_path, 'ml-1m/movies.dat')
users_file = os.path.join(drive_path, 'ml-1m/users.dat')
ratings_file = os.path.join(drive_path, 'ml-1m/ratings.dat')

#Load
movies = pd.read_csv(movies_file, sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv(users_file, sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv(ratings_file, sep = '::', header = None, engine = 'python', encoding = 'latin-1')

# Display
print(' MOVIES', movies.head(3), '\n\n', 'USERS', users.head(3), '\n\n', 'RATINGS', ratings.head(3))

 MOVIES    0                        1                             2
0  1         Toy Story (1995)   Animation|Children's|Comedy
1  2           Jumanji (1995)  Adventure|Children's|Fantasy
2  3  Grumpier Old Men (1995)                Comedy|Romance 

 USERS    0  1   2   3      4
0  1  F   1  10  48067
1  2  M  56  16  70072
2  3  M  25  15  55117 

 RATINGS    0     1  2          3
0  1  1193  5  978300760
1  1   661  3  978302109
2  1   914  3  978301968


In [80]:
# Load training and test sets

# Paths
training_set_file = os.path.join(drive_path, 'ml-100k/u1.base')
test_set_file = os.path.join(drive_path, 'ml-100k/u1.test')

# Load
# 0th is row, 1st column = user, 2nd = movie, 3rd = rating, 4th = timestamp
training_set = pd.read_csv(training_set_file, delimiter = '\t')
test_set = pd.read_csv(test_set_file, delimiter = '\t')

# Display
print(' TRAINING', training_set.head(3), '\n\n', 'TESTING', test_set.head(3))

# Turn to arrays (same values still)
training_set = np.array(training_set, dtype = 'int')
test_set = np.array(test_set, dtype = 'int')

 TRAINING    1  1.1  5  874965758
0  1    2  3  876893171
1  1    3  4  878542960
2  1    4  3  876893119 

 TESTING    1   6  5  887431973
0  1  10  3  875693118
1  1  12  5  878542960
2  1  14  5  874965706


In [81]:
# Get total number of users and movies, across
# Gives total across train and test data (cross-validation)
nb_users = int(max(max(training_set[:,0]), max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]), max(test_set[:,1])))

In [82]:
# Convert data into array with a user on each line and movies in columns
def convert(data):
    # Create list of list. Each list corresponds to a user, and their movie ratings
    new_data = []
    # Add ratings into user's list
    for id_users in range(1, nb_users + 1):
      id_movies = data[:, 1][data[:, 0] == id_users]
      id_ratings = data[:, 2][data[:, 0] == id_users]
      # Fill with zeros
      ratings = np.zeros(nb_movies)
      # Replace zeros with real ratings
      ratings[id_movies - 1] = id_ratings
      new_data.append(list(ratings))
    return new_data

# Contains 943 rows of lists. In each list is the user's ratings of each movie
# Moves without a rating just have a 0
training_set = convert(training_set)
test_set = convert(test_set)

In [83]:
# Convert data into Torch tensors to enable manipulation in PyTorch
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

Up to here, all the data preprocessing could be used for other types of models. After here, it's specific to Botlzmann models

In [84]:
# Convert ratings into binary (1 = liked, 0 = not liked, -1 = no rating)
training_set[training_set == 0] = -1
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

## Boltzmann model

In [85]:
class RBM():
  # Self reference, visible nodes, hidden nodes
  def __init__(self, nv, nh):
    # Initialise weights
    self.W = torch.randn(nh, nv)
    # Probability of hidden nodes, given visible nodes. 1 = batch, nh = bias
    self.a = torch.randn(1, nh)
    # Bias for visible nodes
    self.b = torch.randn(1, nv)
  # Calculate probability that hidden neuron h = 1 given the visible neuron
  def sample_h(self, x):
    # Make product of two tensors. 'mm' does this for two torch tensors. x = visible neuron, w = tensor of weights
    wx = torch.mm(x, self.W.t())
    # Activation function. wx + bias. Apply bias to each line of mini-batch using expand_as
    activation = wx + self.a.expand_as(wx)
    # Probability that hidden node is activated, given visible node. Calculated as sigmoid of activation
    p_h_given_v = torch.sigmoid(activation)
    # If random number is below 70%, activate neuron, otherwise don't. Gives 0 & 1
    return p_h_given_v, torch.bernoulli(p_h_given_v)
  # Calculate probability that visible neuron v = 1 given the hidden neuron
  def sample_v(self, y):
    # Transpose not needed. W is weight matrix of pv given h so you need transpose for ph given v. Here it's just pv given h though so no transpose
    wy = torch.mm(y, self.W)
    # b, not a
    activation = wy + self.b.expand_as(wy)
    p_v_given_h = torch.sigmoid(activation)
    # From vector of probabilities, give some sampling. If random number from sampling is below 0.25, give 1, otherwise 0
    # Depending on 0 or 1, this is the prediction of whether or not user will give a like
    return p_v_given_h, torch.bernoulli(p_v_given_h)
# v0 = input vector containing ratings of all the movies by one user
# vk = visible nodes obtained after k-sampling
# ph0 = vector of probabilities which at first iteration have hidden nodes h = 1, given the values of v0
# phk = probability of hidden nodes after k-sampling given values of visible nodes vk
# Other parameters like learning rate could be added to improve model
  def train(self, v0, vk, ph0, phk):
    self.W += (torch.mm(v0.t(), ph0) - torch.mm(vk.t(), phk)).t()
    self.b += torch.sum((v0 - vk), 0)
    self.a += torch.sum((ph0 - phk), 0)
# Nodes are ratings of all movies by a user in this example so NV (number of visible nodes) is the number of movies
nv = len(training_set[0])
# nh can be any number but there are 1,682 movies. nh corresponds to actual features so maybe 100 features. Best to tune and try different numbers
nh = 100
# Also tuneable
batch_size = 100
# Create RBM object
rbm = RBM(nv, nh)

RBM is an energy-based model i.e., you're trying to minimize the energy function for a normal state. Energy depends on weighting of the model based on the tensors.

Weights can be optimized to minimize the energy of the model. Minimizing the energy is equivalent to maximizing the log-likelihood gradient of the training set, so that's what you're computing in the model.

Computing the log-likelihood gradient of the training set directly is too computationally expensive, so instead you can reach it through better and better approximations. Tiny adjustments in the direction of minimal energy.

Contrastive divergence learning allows you to get those adjustments. You can do this using a Gibbs chain in k number of steps. K * hidden nodes and visible nodes. So given v0, sample the probable hidden nodes for v0 and then use those probable hidden nodes to sample the probable visible nodes. Repeat for v1, v2...vK. That's a CDK algorithm: k-step contrastive divergence.

In [86]:
# Training RBM model

nb_epoch = 10
for epoch in range(1, nb_epoch + 1):
  # Need loss function to measure error between predictions and observations
  # Could use RMSE (root mean squared error), but also simple distance or absolute distance
  # Loss starts at 0 and increases with errors
  train_loss = 0
  # Also need a counter to normalize train_loss by dividing train_loss by counter
  s = 0.
  # Update weights in batches of users, not per each user. Batch size 100
  for id_user in range(0, nb_users - batch_size, 100):
    # Input is vector going into Gibbs chain. Ratings of all the users in the batch
    vk = training_set[id_user : id_user + batch_size]
    # Movie ratings from batch. Same at start, will be updated
    v0 = training_set[id_user : id_user + batch_size]
    # Need to get initial probabilities. Ph0 = probabilities that h nodes = 1 at start, given user ratings
    ph0,_ = rbm.sample_h(v0)
    # For loop takes you through samples of hidden and visible nodes. Goes until 10th sample
    for k in range(10):
      # Hidden nodes obtained at k-th step of contrastive divergence
      _,hk = rbm.sample_h(vk)
      # First update of visible nodes after random sampling
      _,vk = rbm.sample_v(hk)
      # Some cells contain no ratings (coded as -1). To prevent -1 ratings from interfering with
      # updating process, do this to keep -1 ratings:
      vk[v0 < 0] = v0[v0 < 0]
    # Apply train function to update weights & bias
    phk,_ = rbm.sample_h(vk)
    rbm.train(v0, vk, ph0, phk)
    # Update train loss, using absolute difference between predictions and actual values
    train_loss += torch.mean(torch.abs(v0[v0 >= 0] - vk[v0 >= 0]))
    s += 1.
  print('epoch: ' + str(epoch) + '  loss: ' + str(train_loss / s))

epoch: 1  loss: tensor(0.3273)
epoch: 2  loss: tensor(0.2490)
epoch: 3  loss: tensor(0.2497)
epoch: 4  loss: tensor(0.2503)
epoch: 5  loss: tensor(0.2489)
epoch: 6  loss: tensor(0.2494)
epoch: 7  loss: tensor(0.2472)
epoch: 8  loss: tensor(0.2460)
epoch: 9  loss: tensor(0.2485)
epoch: 10  loss: tensor(0.2488)


Better attempt at explaining a Gibbs chain:

If you've got a dataset with related variables, there should be some kind of 'typical' configuration of the variables which you could infer from the actual values. Like a normal distribution across multiple variables, taking into account how they interact with each other.

When you're looking at your data for the first time, you don't know exactly what that idealized configuration of the variables would look like, but you can figure it out using a Gibbs chain.

To do that, you could take a row of data from the table \(let's say values v1, v2, and v3 from three variables). Blank out v3, and see what v1 and v2 suggest that v3 is likely to be. Then update v3, blank out v1, and use the new v3 and v2 to predict v1. Then use updated v3 and v2 to predict v1 and so on. You keep doing this round and round until you reach a resting state where the variables stop updating in any significant way.

That state represents \(roughly) the most likely state that you'll probably see the variables distributed in, given the relationships between them.

In [87]:
# Testing RBM
# Code essentially being reused from above minus a few small cuts and edits for testing, not training

test_loss = 0
s = 0.
for id_user in range(nb_users):
  # v is input on which you're making the prediction
  v = training_set[id_user : id_user + 1]
  # vt is target, not previous data
  vt = test_set[id_user : id_user + 1]
  if len(vt[vt >= 0]) > 0:
    # No k steps
    _,h = rbm.sample_h(v)
    _,v = rbm.sample_v(h)
    # Still take absolute distance between prediction and target
    test_loss += torch.mean(torch.abs(vt[vt >= 0] - v[vt >= 0]))
    s += 1.
print('Test loss: ' + str(test_loss / s))

Test loss: tensor(0.2455)


When testing the RBM model, you don't need the for loop to run over 10 ephochs because you're only trying to predict a single value. Since you're only making one prediction instead of 10, in theory it should be an accurate prediction \(relates to MCMC)

Model can now predict binary ratings \(0 or 1) pretty accurately